# Encoding message in an image

In [ ]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

def modify_selected_channels(pixel_rgb, message_bits, bits_per_channel, channels_to_encode):
    expected_length = len(channels_to_encode) * bits_per_channel
    message_bits = message_bits.ljust(expected_length, '0')

    modified_pixel = list(pixel_rgb)
    bit_idx = 0

    for channel in channels_to_encode:
        bits = message_bits[bit_idx:bit_idx + bits_per_channel]
        bit_idx += bits_per_channel

        channel_idx = 'RGB'.index(channel)
        value = modified_pixel[channel_idx]

        # mask to clear the LSBs
        mask = 0xFF ^ ((1 << bits_per_channel) - 1)
        cleared = value & mask

        # Insert new bits
        modified_pixel[channel_idx] = cleared | int(bits, 2)

    return tuple(modified_pixel)


def encode_message_in_image(img_path, out_path, message, start_position=(0, 0), gap=0,
                            channels='RGB', num_bits=1, delimiter_start='#', delimiter_end='#',
                            horizontal=1):
    # checking channel input
    valid_channels = set('RGB')
    input_channels = set(channels.upper())
    if not input_channels.issubset(valid_channels):
        print("!!! Invalid channel input. Use only 'R', 'G', 'B'.")
        return

    # checking num_bits
    if not (1 <= num_bits <= 8):
        print("!!! Invalid num_bits. num_bits must be between 1 and 8.")
        return

    # checking horizontal input
    if horizontal not in [0, 1]:
        print("!!! Invalid horizontal value. It should be either 0 or 1.")
        return

    img = np.array(Image.open(img_path))
    rows, cols, _ = img.shape

    # checking start_position
    r_start, c_start = start_position
    if r_start >= rows or c_start >= cols:
        print(f"!!! Invalid start_position. The start position {start_position} exceeds image dimensions ({rows}, {cols}).")
        return

    channels = channels.upper()

    # Converting message to binary
    binary_message = ''.join(format(ord(c), '08b') for c in message)

    #converting delimiters to binary
    if delimiter_start:
        binary_start = ''.join(format(ord(c), '08b') for c in delimiter_start)
        binary_message = binary_start + binary_message

    if delimiter_end:
        binary_end = ''.join(format(ord(c), '08b') for c in delimiter_end)
        binary_message += binary_end

    # Calculate bits per pixel
    bits_per_pixel = len(channels) * num_bits

    # Calculate available pixels
    total_pixels = ((rows - r_start - 1) * cols + (cols - c_start)) if horizontal else ((cols - c_start - 1) * rows + (rows - r_start))
    available_pixels = total_pixels // (gap + 1) + (1 if total_pixels % (gap + 1) != 0 else 0)
    available_bits = available_pixels * bits_per_pixel
    max_characters = available_bits // 8

    # Checking if the message fits
    if len(binary_message) > available_bits:
        print(f"!!! Error: Message is too large to fit in the image. Max bits available: {available_bits}")
        print(f"   ➤ Message length (in bits): {len(binary_message)}")
        return

    # Pad binary message to match required length
    if len(binary_message) % bits_per_pixel != 0:
        binary_message = binary_message.ljust(
            ((len(binary_message) // bits_per_pixel) + 1) * bits_per_pixel,
            '0'
        )

    # Encoding process
    bit_pointer = 0
    r, c = r_start, c_start

    if horizontal:
        for r in range(r_start, rows):
            for c in range(c, cols, gap + 1):
                if bit_pointer >= len(binary_message):
                    break

                pixel = img[r, c]
                message_bits = binary_message[bit_pointer: bit_pointer + bits_per_pixel]
                modified_pixel = modify_selected_channels(
                    pixel, message_bits, num_bits, channels
                )

                img[r, c] = modified_pixel
                bit_pointer += bits_per_pixel

            if c >= cols - gap - 1:
                c = (c + gap + 1) % cols

            if bit_pointer >= len(binary_message):
                break

    else:  # Vertical encoding
        for c in range(c_start, cols):
            for r in range(r, rows, gap + 1):
                if bit_pointer >= len(binary_message):
                    break

                pixel = img[r, c]
                message_bits = binary_message[bit_pointer: bit_pointer + bits_per_pixel]
                modified_pixel = modify_selected_channels(
                    pixel, message_bits, num_bits, channels
                )

                img[r, c] = modified_pixel
                bit_pointer += bits_per_pixel

            if r >= rows - gap - 1:
                r = (r + gap + 1) % rows

            if bit_pointer >= len(binary_message):
                break

    # Save the modified image
    Image.fromarray(img).save(out_path)
    print(f" Message successfully encoded and saved to '{out_path}'")


# Example usage
encode_message_in_image(
    img_path='man.jpg',
    out_path='enc_man.png',           # Output as PNG
    message="hi guys iam from datascience"*100,
    start_position=(0, 0),
    gap=3,
    channels='RGB',
    num_bits=6,
    delimiter_start='#',
    delimiter_end='#',
    horizontal=1
)

In [ ]:
i1 = np.array(Image.open('man.jpg'))
i2 = np.array(Image.open('enc_man.png'))

plt.subplot(1, 2, 1)
plt.imshow(i1)
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(i2)
plt.axis('off')

plt.show()
